In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('hh_ru_dataset.csv')

В целом смотрим только на следующие столбцы: final_state, work_experience_months, education_level, expected_salary, compensation_from, compensation_to в срезе profession

In [ ]:
# отбрасываем дубликаты
# df = df[['profession', 'final_state', 'expected_salary', 'compensation_from', 'compensation_to'
#          , 'work_experience_months', 'education_level']]
df = df.drop_duplicates()
df = df[(~df.profession.isna())&(~df.expected_salary.isna())&(~df.education_level.isna())&(~df.final_state.isna())&(~df.compensation_from.isna()) & (~df.compensation_to.isna()) & (~df.work_experience_months.isna())]
df = df[(df.compensation_to != 0)&(df.expected_salary!=0)]

In [ ]:
# compensation_from - убирать нули и выбросы?
df = df[df.work_experience_months <= (df.year_of_birth + 1 - 14)*12]

In [ ]:
# отбрасывание статусов response-response, т.к. не знаем что с ними делать
df = df[df.final_state!='response']

Новые столбцы

In [ ]:
df['final_state'] = df['final_state'].map(lambda x: int(x=='invitation'))

In [ ]:
edu_dict = {
    'secondary' : 0,
    'special_secondary': 1,
    'unfinished_higher':2,
    'bachelor':3,
    'higher':3,
    'master':4,
    'candidate':4,
    'doctor':4
}
df['education_level'] = df['education_level'].map(lambda x: edu_dict[x])

In [ ]:
management = ['Менеджер по продажам, менеджер по работе с клиентами', 'Бухгалтер', 'Менеджер по маркетингу, интернет-маркетолог',
             'Менеджер по персоналу', 'Менеджер по логистике, менеджер по ВЭД', 'Специалист по подбору персонала',
             'Менеджер по закупкам', 'Директор магазина, директор сети магазинов', 'Руководитель отдела продаж',
             'Офис-менеджер', 'Секретарь, помощник руководителя, ассистент', 'Руководитель проектов',
             'Финансовый менеджер', 'Администратор', 'Продавец-консультант, продавец-кассир', 
             'Менеджер по туризму', 'Координатор отдела продаж', 'Менеджер ресторана', 'Специалист по кадрам',
             'Менеджер/руководитель АХО']
design = ['Дизайнер, художник']
it = ['Программист, разработчик', 'Оператор ПК, оператор базы данных', 'Системный администратор',
     'SMM-менеджер, контент-менеджер', 'Тестировщик', 'Аналитик']
service = ['Официант, бармен, бариста', 'Специалист технической поддержки', 'Водитель', 'Охранник',
          'Курьер', 'Супервайзер', 'Кассир-операционист', 'Диспетчер', 'Торговый представитель', 
          'Упаковщик, комплектовщик', 'Повар, пекарь, кондитер', 'Кладовщик', 
          'Продавец-консультант, продавец-кассир', 'Начальник склада', 'Фитнес-тренер, инструктор тренажерного зала',
          'Товаровед', 'Оператор call-центра, специалист контактного центра', 'Уборщица, уборщик',
          'Разнорабочий', 'Мерчандайзер']
law = ['Делопроизводитель, архивариус', 'Юрисконсульт', 'Экономист', 'Юрист', 'Кредитный специалист']
engineering = ['Инженер-энергетик, инженер-электрик', 'Сварщик', 'Инженер-конструктор, инженер-проектировщик',
              'Слесарь, сантехник', 'Начальник производства', 'Начальник смены, мастер участка', 
              'Оператор производственной линии', 'Технолог', 'Машинист', 'Механик', 'Сервисный инженер, инженер-механик',
              'Инженер по охране труда и технике безопасности, инженер-эколог']
teacher = ['Учитель, преподаватель, педагог', 'Воспитатель, няня']
med = ['Психолог', 'Врач']
def f(x):
    if x in management:
        return "Менеджмент"
    if x in design:
        return "Дизайн"
    if x in it:
        return "IT"
    if x in service:
        return "Сфера услуг"
    if x in law:
        return "Право/Экономика"
    if x in engineering:
        return "Инженерия/Промышленность"
    if x in teacher:
        return "Образование"
    return "Медицина"
df['occupation'] = df.profession.map(f)

In [ ]:
d = dict()
qs = dict()
compensation_qs = dict()
for i in df.profession.unique():
    temp = df[df.profession==i]
    t1 = temp.groupby('resume_id').expected_salary.mean()
    q1, q2 = t1.quantile(0.05), t1.quantile(0.95)
    if q1 < 1000:
        print(i)
    if q2 > 1000000:
        print(i, q2)
    qs[i] = (q1, q2)
    t2 = temp.groupby('vacancy_id').compensation_to.mean()
    q1, q2 = t2.quantile(0.05), t2.quantile(0.95)
    if q1 < 1000:
        print(i)
    if q2 > 1000000:
        print(i, q2)
    compensation_qs[i] = (q1, q2)
    
def app(x):
    q = qs[x['profession']]
    q1 = compensation_qs[x['profession']]
    a = (q[0] <= x['expected_salary']) and (x['expected_salary'] <= q[1])
    b = (q1[0] <= x['compensation_to']) and (x['compensation_to'] <= q1[1])
    return  a and b
df = df[df.apply(app, 1)]
    
    
for i in df.profession.unique():
    d[i] = ((temp.groupby('vacancy_id').compensation_to.mean() + temp.groupby('vacancy_id').compensation_from.mean())/2).median() #медиана средних зп
def temp(x):
    t = d[x['profession']]
    return (x['expected_salary'] - t) / t
df['delta_salary'] = df.apply(temp, 1)

In [ ]:
df.to_csv('new_normal.csv')

In [ ]:
# делаем модель
import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
import statsmodels.stats.tests.test_influence
import numpy as np

In [ ]:
def show_cat(cat, print_sum=False):
    temp = df[df.profession.map(lambda x: x in cat)]
    x = temp[['delta_salary', 'education_level', 'work_experience_months']]
    x1 = sm.add_constant(x)
    y = temp['final_state']
    model_1 = sm.GLM(
        y,
        x1,
        family=families.Binomial(),
    ).fit()
    if print_sum:
        print(model_1.summary())
    ans = str(model_1.summary()).splitlines()[15][61:66]
    return float(ans)
    # print(model_1.normalized_cov_params)
    # return model_1.pvalues['delta_salary'], model_1.normalized_cov_params['delta_salary']

In [ ]:
res = pd.DataFrame({}, columns=['profession', 'p-value', 'mean_salary', 'df_counts', 'ok'])
for i in df.profession.unique():
    t = show_cat([i])
    if t > 0.05:
        res=res.append({'profession':i, 'p-value':t,
                   'mean_salary':d[i], 'df_counts':(df.profession==i).sum(),
                       'ok':(t>0.05)}, ignore_index=True)
        # print(i, t, d[i], len(df[df.profession==i]), sep='|\t')
res.columns = ['Специадбность', 'p-значение', 'Средняя зп для специальности', 'Количество наблюдений', 'Значимость']
res